<a href="https://colab.research.google.com/github/SkondoR-DA/Dota2-match-predict/blob/main/Pars_pub_match.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests
import time
import json
from pathlib import Path

# Задачи:
1. ~~Убрать дубликаты матчей~~
2. ~~Перевести время (start_time) и добавить разделитель в duration.~~ Разбить id героев по столбцам
3. ~~Поднять PostgreSQL~~ и добавить загрузку
4. Автоматизировать (Cron)


In [2]:
# Путь к файлу
filepath = Path('/content/drive/MyDrive/df.csv')

In [3]:
# Загрузка файла
df = pd.read_csv(filepath)

In [4]:
df

,Unnamed: 0,match_id,match_seq_num,radiant_win,start_time,duration,avg_mmr,num_mmr,lobby_type,game_mode,avg_rank_tier,num_rank_tier,cluster,radiant_team,dire_team
0,0,6833937813,5724027212,True,1667190893,1502,3167.0,1.0,7,22,81,3,185,"136,86,94,11,135","9,71,44,36,119"
1,1,6833934314,5724024879,False,1667190637,1570,NaN,NaN,7,22,81,5,151,"5,88,93,100,17","52,35,38,9,62"
2,2,6833945514,5724035254,False,1667191374,1549,5063.0,5.0,7,3,81,9,227,"28,64,95,101,62","9,19,112,44,39"
3,3,6833938207,5724033627,False,1667190914,1924,4830.0,2.0,7,22,81,3,154,"74,41,105,75,59","25,93,107,120,87"
4,4,6833945600,5724038589,False,1667191395,1797,4382.0,3.0,7,22,81,4,251,"41,97,75,76,44","21,106,70,62,30"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,165,6838597809,5728232826,False,1667393390,1606,3076.0,1.0,7,22,81,4,184,"135,128,12,100,19","85,14,11,123,106"
166,166,6838630617,5728250011,False,1667394333,1210,NaN,NaN,7,22,81,4,273,"70,26,62,135,25","92,137,136,86,19"
167,167,6838609218,5728253223,False,1667393724,1890,3092.0,2.0,7,22,81,5,182,"26,100,82,19,44","11,86,129,48,3"
168,23,6838639500,5728260916,True,1667394582,1295,5003.0,4.0,7,22,81,8,153,"30,100,93,136,106","71,120,9,89,84"


In [5]:
# Дроп колонки индекса от предыдущего сохранения. !!! Переделать сохранение index=False
df = df.drop(df.columns[[0]], axis=1)
df.head(1)

,match_id,match_seq_num,radiant_win,start_time,duration,avg_mmr,num_mmr,lobby_type,game_mode,avg_rank_tier,num_rank_tier,cluster,radiant_team,dire_team
0,6833937813,5724027212,True,1667190893,1502,3167.0,1.0,7,22,81,3,185,"136,86,94,11,135","9,71,44,36,119"


In [6]:
# API запрос с переводом json'а в датафрейм
j = requests.get(f'https://api.opendota.com/api/publicMatches')
publicMatches = j.json()
df_publicmatches = pd.DataFrame.from_dict(publicMatches)

In [7]:
# Фильтрация высокоранговых матчей. Проверка количетсва строк
df_publicmatches_hight_mmr = df_publicmatches.query('avg_rank_tier == 81')
df_publicmatches_hight_mmr.shape

(3, 14)

In [8]:
# Присоединения новых строк к датафрейму. Проверка количества строк
frames = [df, df_publicmatches_hight_mmr]
df = pd.concat(frames)
df.shape

(173, 14)

In [9]:
# Создание массива и датафрейма с уникальными match_id для более сложного API запроса с более полной статистикой по матчу. Требует доработки
# matches_array = df.match_id.unique()
# matches_array = pd.DataFrame.from_dict(matches_array)
# df_match_id = matches_array.rename(columns={ 0 : "match_id"})
# df_match_id.shape

In [10]:
# Так как API запрос парсит недавние 100 матчей, рандомно. То могут встречаться дубликаты. Избавимся от них
df = df.drop_duplicates(subset=['match_id'])

In [11]:
# Сохранение 
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath)

In [12]:
# Перевод в формат времени
# df['start_time'] = pd.to_datetime(df['start_time'], unit='s')
# df['duration'] = pd.to_datetime(df['duration'], unit="s").dt.time